In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import pandas as pd
import os
import zipfile
from sklearn.model_selection import train_test_split

# Ensure keras-tuner is installed
!pip install keras-tuner -q

!pip install kaggle

print("Upload your kaggle.json file:")
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("Downloading UTKFace dataset...")
!kaggle datasets download -d jangedoo/utkface-new
print("Download complete.")

print("Unzipping...")
with zipfile.ZipFile('utkface-new.zip', 'r') as zip_ref:
    zip_ref.extractall('utkface_dataset')
print("Unzip complete.")

# Data loading and preprocessing
image_dir = 'utkface_dataset/UTKFace'
filenames = os.listdir(image_dir)

files_list = []
genders_list = []

for f in filenames:
    try:
        gender = f.split('_')[1]
        files_list.append(f)
        genders_list.append(gender)
    except Exception as e:
        pass

df = pd.DataFrame({'filename': files_list, 'gender': genders_list})

df['gender'] = df['gender'].astype(str)

train_df, validation_df = train_test_split(df, test_size=0.2, random_state=42)

print(f"Total images: {len(df)}")
print(f"Training images: {len(train_df)}")
print(f"Validation images: {len(validation_df)}")

# Image Data Generators
IMG_SIZE = (128, 128)

# Training Generator (using Data Augmentation)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Validation Generator
validation_datagen = ImageDataGenerator(rescale=1./255)

# Load images using dataframe
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=image_dir,
    x_col='filename',
    y_col='gender',
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_dataframe(
    dataframe=validation_df,
    directory=image_dir,
    x_col='filename',
    y_col='gender',
    target_size=IMG_SIZE,
    batch_size=32,
    class_mode='binary'
)

print(train_generator.class_indices)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 11.7 MB/s eta 0:00:00
Upload your kaggle.json file:


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
 91% 300M/331M [00:04<00:00, 55.7MB/s]
100% 331M/331M [00:04<00:00, 71.6MB/s]
Download complete.
Unzipping...
Unzip complete.
Total images: 23708
Training images: 18966
Validation images: 4742
Found 18966 validated image filenames belonging to 2 classes.
Found 4742 validated image filenames belonging to 2 classes.
{'0': 0, '1': 1}


In [ ]:
# Define the model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification for gender
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     1,605,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,699,009 (6.48 MB)

 Trainable params: 1,699,009 (6.48 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# Train the model directly
print("Starting model training...")
history = model.fit(
    train_generator,
    epochs=30,
    validation_data=validation_generator,
    callbacks=[early_stopping]
)
print("Training complete.")

# Display the model summary
model.summary()

Starting model training...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 110s 171ms/step - accuracy: 0.6263 - loss: 0.6493 - val_accuracy: 0.7560 - val_loss: 0.5008
Epoch 2/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 95s 159ms/step - accuracy: 0.7194 - loss: 0.5584 - val_accuracy: 0.7676 - val_loss: 0.4844
Epoch 3/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 93s 157ms/step - accuracy: 0.7383 - loss: 0.5302 - val_accuracy: 0.7864 - val_loss: 0.4463
Epoch 4/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 94s 159ms/step - accuracy: 0.7582 - loss: 0.5020 - val_accuracy: 0.7995 - val_loss: 0.4377
Epoch 5/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 94s 158ms/step - accuracy: 0.7730 - loss: 0.4709 - val_accuracy: 0.8121 - val_loss: 0.4004
Epoch 6/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 94s 158ms/step - accuracy: 0.7826 - loss: 0.4608 - val_accuracy: 0.8205 - val_loss: 0.3875
Epoch 7/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 95s 160ms/step - accuracy: 0.8009 - loss: 0.4259 - val_accuracy: 0.8049 - val_loss: 0.4601
Epoch 8/30
593/593 ━━━━━━━━━━━━━━━━━━━━ 92s 156ms/step - accuracy: 0.8145 - loss: 

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │     1,605,696 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,097,029 (19.44 MB)

 Trainable params: 1,699,009 (6.48 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,398,020 (12.96 MB)

In [ ]:
# Changed to save the directly trained model
print("Saving the trained model...")
model.save("final_gender_model.h5")

print("Model saved as 'final_gender_model.h5'")

Saving the trained model...
Model saved as 'final_gender_model.h5'


In [ ]:
#Without Image display

import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import files


print("Loading Final Gender model...")
model = tf.keras.models.load_model("final_gender_model.h5")
print("Model loaded successfully!")

print("\nPlease upload an image to predict:")
uploaded = files.upload()

for filename in uploaded.keys():
    img_path = filename

    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    img_array_preprocessed = img_array / 255.0


    prediction = model.predict(img_array_preprocessed)[0][0]

    print("\n--- Prediction Result ---")
    print(f"Raw Prediction Value: {prediction}")

    if prediction > 0.5:
        print("Predicted: Female (Class 1)")
    else:
        print("Predicted: Male (Class 0)")

Loading Final Gender model...
Model loaded successfully!

Please upload an image to predict:


Saving photo_2023-08-12_16-14-35 (2).jpg to photo_2023-08-12_16-14-35 (2).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 778ms/step

--- Prediction Result ---
Raw Prediction Value: 0.02344149351119995
Predicted: Male (Class 0)


In [ ]:
#with Display image

import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
from google.colab import files
from IPython.display import display, Image # Import functions to display images

print("Loading Final Gender model...")
model = tf.keras.models.load_model("final_gender_model.h5")
print("Model loaded successfully!")

print("\nPlease upload an image to predict:")
uploaded = files.upload()

for filename in uploaded.keys():
    img_path = filename

    # --- Display the uploaded image ---
    print(f"\n Displaying: {filename}")
    display(Image(filename=img_path, width=200)) # Set a width for better display
    # ------------------------------------

    # Preprocess the image for the model
    img = image.load_img(img_path, target_size=(128, 128))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)

    img_array_preprocessed = img_array / 255.0

    # Make the prediction
    prediction = model.predict(img_array_preprocessed)[0][0]

    print("\nPrediction Result")

    if prediction > 0.5:
        print("Predicted: Female (Class 1)")
    else:
        print("Predicted: Male (Class 0)")